# Import libraries

In [2]:
import cobra
import requests
import pandas as pd
import numpy

In [2]:
add_kegg_id = False

# Add KEGG ID in model

Note : CSV must contain one column 'Reaction name', one 'KEGG reaction ID' and one 'Gene name'

In [3]:
if add_kegg_id:
    model_to_improve = cobra.io.read_sbml_model("Models/ccm_ross.xml")

In [24]:
# metabolites_list = [metabolite.id for metabolite in model.metabolites]

In [4]:
# metabolites_list

In [5]:
df_react = pd.read_csv("tab_reactions.csv", sep=";")
# df_react = df_react.dropna()
df_react = df_react.set_index('Reaction name')
df_react

,KEGG reaction ID,Gene name
Reaction name,,
pts,RPTSsy,"ptsG,ptsH,ptsI"
pgi,R02740,pgi
pfk,R04779,pfkA
fbp,R00762,fbp
ald,R01070,fbaA
tim,R01015,tpiA
gap,R01061,"gapA, gapB"
pgk,R01512,pgk
pgm,R01518,gpml/pgml


In [6]:
def add_kegg_id_to_model(model, dataframe_react):
    for reaction in model.reactions:
        print(reaction.id)
        try :
            if dataframe_react.loc[reaction.id]['KEGG reaction ID'] != "NaN":
                reaction.notes = {"kegg_id": dataframe_react.loc[reaction.id]['KEGG reaction ID']}
            else:
                print(dataframe_react.loc[reaction.id]['KEGG reaction ID'])
        except :
            print("no")

In [7]:
if add_kegg_id:
    add_kegg_id_to_model(model_to_improve, df_react)
    cobra.io.write_sbml_model(model_to_improve, filename="model_file.xml")

# Get AA sequence of enzyme

In [8]:
new_model = cobra.io.read_sbml_model("Models/model_ccm_2.xml")
new_model

Name,ccm_ross_xml
Memory address,0x022f397b44c0
Number of metabolites,60
Number of reactions,69
Number of groups,0
Objective expression,1.0*biomass - 1.0*biomass_reverse_01e59
Compartments,"ext, int"


### Add gene reaction rule to the model
Takes as input a dictionary that links the reaction IDs of the model to the gene IDs.

In [3]:
data = pd.read_csv("tab_reactions.csv", sep=",")
model = cobra.io.read_sbml_model("./model_ccm_2.xml")

In [4]:
from collections import defaultdict
reaction2genes = defaultdict(list)
for i in range(len(data)):
    reaction_name_tmp = data.loc[i,"Reaction name"]

    if not isinstance(reaction_name_tmp,str) :
        pass
    else:
        reaction_name = reaction_name_tmp

    if str(data.loc[i,"Gene name"]) != "nan":
        if "?" in data.loc[i,"Gene name"]:
            continue
        elif "/" in data.loc[i,"Gene name"]:
            genes = data.loc[i,"Gene name"].split("/")
            print(genes, type(genes))
            for gene_name in genes :
                reaction2genes[reaction_name].append(gene_name)
        elif "(" in data.loc[i,"Gene name"] :
            gene = data.loc[i,"Gene name"].split("(")[0].strip(" ")
            reaction2genes[reaction_name].append(gene)
        else:
            reaction2genes[reaction_name].append(data.loc[i,"Gene name"])
    else : continue

['gpml', 'pgml'] <class 'list'>
['ActP', 'staP'] <class 'list'>


In [5]:
for reac_id, genes in reaction2genes.items():
    try:

        reac_obj = model.reactions.get_by_id(reac_id)
        gene_reaction_rule = " or ".join(genes)
        reac_obj.gene_reaction_rule = gene_reaction_rule

    except KeyError:
        continue

In [6]:
enzyme_list = [(enzyme.id, enzyme.notes['kegg_id']) for enzyme in model.reactions if 'kegg_id' in enzyme.notes]

In [12]:
reac = model.reactions.get_by_id("tim")
"tpiA" in [g.id for g in reac.genes]

True

In [24]:
def find_compounds_AAseq(list_of_enzymes, dict_seq = {}):
    for enzyme in list_of_enzymes:
        print(f"Looking for compounds and AA sequence of {enzyme[0]}:{enzyme[1]}")
        aa = False
        ortho = False
        ko = False
        seq = ""
        url = "http://rest.kegg.jp/get/"

        # First request to get list of compounds and the KEGG Orthology ID from the KEGG ID
        r1 = requests.get(url=url+enzyme[1])

        if r1.status_code==200:
            # The content is one long string: splitting with \n to iter on every line
            info = str(r1.content).split('\'')[1].split('\\n')

            for line in info:

                # The line with all compounds:
                if 'EQUATION' in line:
                    compounds = [c for c in line.split() if 'C' in c]

                # Line(s) with KEGG Orthology ID to test
                elif 'ORTHOLOGY' in line or ortho:
                    # print(line)

                    if ortho:
                        id = line.split()[0]
                        if id == 'DBLINKS' or id =='///':
                            ko = True
                            print(f"\n[!]KEGG ID {enzyme[1]} has no corresponding KEGG Orthology ID for organism ece\n\n===\n")
                            break
                    else:
                        id = line.split()[1]
                        ortho = True

                    # Second request to get CDS ID from KEGG ORthology ID
                    r2 = requests.get(url=url+id)

                    if r2.status_code==200:
                        info = str(r2.content).split('\'')[1].split('\\n')

                        for line in info:

                            # Line corresponding to the organism target (e.coli)
                            if 'ECO:' in line:
                                ############# modifier pour verifier les sous unites #############
                                
                                #For each line, we select the ones corresponding to the organism's ID
                                #And store in a list of tuples the KEGG gene IDs and corresponding gene names.
                                #We can then use the gene names to check if the KEGG IDs are the right ones.

                                line_list = line.split(" ")
                                start_index = line_list.index("ECO:")
                                idZ_list = line_list[start_index+1:]
                                idZ_gID = [(idZ, gID) for idZ, gID in zip([elem.split('(')[0] for elem in idZ_list],\
                                                                           [elem.split('(')[1].strip(')') for elem in idZ_list])]
                                
                                #Gene ID comparison between the KEGG response and the model's gene names.
                                for idZ, gID in idZ_gID:
                                    reac_obj = model.reactions.get_by_id(enzyme[0])

                                    if gID.lower() in [str(g.id).lower() for g in reac_obj.genes]:
                                        print(f"\n[>]Found {gID} gene id in reaction {reac_obj.name}. Adding its aa sequence to the dictionary.\n\n===\n")
                                    
                                        # Third request to get AA sequence from CDS ID
                                        r3 = requests.get(url=url+"eco:"+idZ)
                                    

                                        if r3.status_code==200:
                                            info = str(r3.content).split('\'')[1].split('\\n')

                                            for line in info:
                                                if 'AASEQ' in line:
                                                    aa = True
                                                    
                                                elif 'NTSEQ' in line:
                                                    break

                                                elif aa:
                                                    seq += line.split()[0]
                                    else:
                                        continue
                                break

                                    

                if seq != "":
                    dict_seq[enzyme[1]] = (compounds, seq)
                    break
                elif ko:
                    break
                elif ortho:
                    aa = False
                    print(f"Trying another KEGG Orthology ID for {enzyme[0]}:{enzyme[1]}")
        

        else: 
            print(f"\n[!]KEGG ID {enzyme[1]} is not found in kegg.\n\n===\n")

    return dict_seq


In [25]:
# Takes 6min10 - 4min5
dict_km_parameters = find_compounds_AAseq(enzyme_list)
# dict_km_parameters

Looking for compounds and AA sequence of ack:R00315
Trying another KEGG Orthology ID for ack:R00315

[!]KEGG ID R00315 has no corresponding KEGG Orthology ID for organism ece

===

Looking for compounds and AA sequence of acn:R01324

[>]Found acnA gene id in reaction acn. Adding its aa sequence to the dictionary.

===

Looking for compounds and AA sequence of ada:R00228
Trying another KEGG Orthology ID for ada:R00228
Trying another KEGG Orthology ID for ada:R00228
Trying another KEGG Orthology ID for ada:R00228
Trying another KEGG Orthology ID for ada:R00228
Trying another KEGG Orthology ID for ada:R00228

[!]KEGG ID R00228 has no corresponding KEGG Orthology ID for organism ece

===

Looking for compounds and AA sequence of adh:R00754
Trying another KEGG Orthology ID for adh:R00754
Trying another KEGG Orthology ID for adh:R00754
Trying another KEGG Orthology ID for adh:R00754

[>]Found adhE gene id in reaction adh. Adding its aa sequence to the dictionary.

===

Looking for compounds 

In [13]:
list_substrat = []
list_enzyme = []

for enzyme in dict_km_parameters:
    for compound in dict_km_parameters[enzyme][0]:
        list_substrat.append(compound)
        list_enzyme.append(dict_km_parameters[enzyme][1])

In [14]:
len(list_enzyme)

140

In [15]:
len(list_substrat)

140

In [16]:
unique_compounds = numpy.unique(list_substrat).tolist()
# unique_compounds

In [17]:
dict_compounds = {}

for compound in unique_compounds:
    url = "http://rest.kegg.jp/get/"
    r = requests.get(url=url+compound)
    car = str(r.content)[1]
    info = str(r.content).split(car)[1].split('\\n')
    dict_compounds[compound] = info[1].split()[1].split(';')[0]


In [18]:
dict_compounds

{'C00001': 'H2O',
 'C00002': 'ATP',
 'C00003': 'NAD+',
 'C00004': 'NADH',
 'C00005': 'NADPH',
 'C00006': 'NADP+',
 'C00007': 'Oxygen',
 'C00008': 'ADP',
 'C00009': 'Orthophosphate',
 'C00010': 'CoA',
 'C00011': 'CO2',
 'C00022': 'Pyruvate',
 'C00024': 'Acetyl-CoA',
 'C00026': '2-Oxoglutarate',
 'C00033': 'Acetate',
 'C00036': 'Oxaloacetate',
 'C00042': 'Succinate',
 'C00058': 'Formate',
 'C00074': 'Phosphoenolpyruvate',
 'C00080': 'H+',
 'C00084': 'Acetaldehyde',
 'C00085': 'D-Fructose',
 'C00091': 'Succinyl-CoA',
 'C00092': 'D-Glucose',
 'C00111': 'Glycerone',
 'C00117': 'D-Ribose',
 'C00118': 'D-Glyceraldehyde',
 'C00122': 'Fumarate',
 'C00149': '(S)-Malate',
 'C00158': 'Citrate',
 'C00197': '3-Phospho-D-glycerate',
 'C00199': 'D-Ribulose',
 'C00227': 'Acetyl',
 'C00231': 'D-Xylulose',
 'C00236': '3-Phospho-D-glyceroyl',
 'C00311': 'Isocitrate',
 'C00345': '6-Phospho-D-gluconate',
 'C00354': 'D-Fructose',
 'C00390': 'Ubiquinol',
 'C00399': 'Ubiquinone',
 'C00469': 'Ethanol',
 'C00631

In [19]:
import pickle

In [23]:
with open("enzyme.p", "wb") as e:
    enzymes = pickle.dump(list_enzyme, e)

with open("substrat.p", "wb") as s:
    substrats = pickle.dump(list_substrat, s)

with open("compound.p", "wb") as c:
    dict_compounds = pickle.dump(dict_compounds, c)

---
# Omnipath tests

In [26]:
import omnipath

In [27]:
help(omnipath)

Help on package omnipath:

NAME
    omnipath

PACKAGE CONTENTS
    _core (package)
    _misc (package)
    constants (package)
    interactions
    requests

DATA
    __email__ = 'turei.denes@gmail.com'
    __full_version__ = '1.0.6'
    __maintainer__ = 'Michal Klein, Dénes Türei'
    __server_version__ = '0.14.46'
    options = Options(url='https://omnipathdb.org', license=No...rue, num_...

VERSION
    1.0.6

AUTHOR
    Michal Klein, Dénes Türei

FILE
    /home/jmuller/miniconda3/envs/rba/lib/python3.10/site-packages/omnipath/__init__.py


